In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
import gdown
from sklearn.metrics import  confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,  StandardScaler, MinMaxScaler,MultiLabelBinarizer,LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin



url = "https://drive.google.com/drive/folders/1K28d1ufLIZL6WtQIS4CE4H_EZZAzvTqX?usp=drive_link"
gdown.download_folder(url, quiet=True)

['/content/Progetto/train.csv']

# **Nurbek`s data preprocessing**

In [ ]:
label_encoder = LabelEncoder()
train_df=pd.read_csv('/content/Progetto/train.csv')
train_df['Crime_Category'] = label_encoder.fit_transform(train_df['Crime_Category'])
X = train_df.drop(columns=['Crime_Category'])
y = train_df['Crime_Category']
# Split features into numerical and categorical columns
numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns
# Handle missing values
X[numerical_columns] = X[numerical_columns].fillna(X[numerical_columns].median())
X[categorical_columns] = X[categorical_columns].fillna('Unknown')
# Convert categorical features to numeric
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le
    # Scale numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Step 2: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the parameter grid
param_grid = {
    'penalty': ['l1','l2'],
    'C': [0.001, 0.01, 0.1, 1, 10],
    'solver': ['liblinear','saga']
}
# Create a GridSearchCV object
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=2, n_jobs=-1, scoring='accuracy')

In [ ]:
grid_search.fit(X_train, y_train)
# Get the best parameters
best_params = grid_search.best_params_
# Get the best estimator
best_model = grid_search.best_estimator_
# Visualize the Confusion Matrix
y_pred = best_model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
print('\n')
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
#model.score(X_test,y_test)
report=classification_report(y_test,y_pred)
print(report)
print(accuracy_score(y_test,y_pred))
# Print the best parameters and score
best_params = grid_search.best_params_
print(best_params)
